# CS 451 Project Preliminary
## FactOrFake : Fake News Detection Using Machine Learning Algorithms 

### 1. Data Collection 

In [1]:
import pandas as pd
import itertools
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_curve, roc_auc_score, precision_score

In [12]:
df_poltifake = pd.read_csv("Datasets\FakeNewsNet\dataset\politifact_fake.csv")
df_poltifake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [13]:
df_poltifake.shape

(432, 4)

In [14]:
df_poltifake['news_url'].nunique()

428

In [15]:
df_poltifake['target'] = 0

In [16]:
df_poltifake.head()

,id,news_url,title,tweet_ids,target
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,0


In [17]:
df_poltifake.drop(df_poltifake.columns[[0,3]],axis=1,inplace=True)

In [18]:
df_poltifake.head()

,news_url,title,target
0,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,0
1,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,0
2,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,0
3,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,0
4,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,0


In [22]:
df_poltifake.dropna()
df_poltifake.shape

(432, 3)

In [23]:
df = pd.read_csv("Datasets\FakeNewsNet\dataset\politifact_real.csv")
df.dropna(inplace=True)
df['target'] = 1
df.drop(df.columns[[0,3]],axis=1,inplace=True)
df.head()

,news_url,title,target
0,http://www.nfib-sbet.org/,National Federation of Independent Business,1
1,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,1
4,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",1
5,http://www.politifact.com/truth-o-meter/statem...,Donald Trump exaggerates when he says China ha...,1
6,https://www.law.cornell.edu/constitution/amend...,25th Amendment,1


In [24]:
df.shape

(373, 3)

In [25]:
df_combined = pd.concat([df_poltifake,df])
df_combined.shape

(805, 3)

In [26]:
df = pd.read_csv("Datasets\FakeNewsNet\dataset\gossipcop_real.csv")
df.head()

,id,news_url,title,tweet_ids
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...
3,gossipcop-857248,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,868114761723936769\t868122567910936576\t868128...
4,gossipcop-884684,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,915528047004209152\t915529285171122176\t915530...


In [27]:
df.dropna(inplace=True)
df['target'] = 1
df.drop(df.columns[[0,3]],axis=1,inplace=True)
df.head()

,news_url,title,target
0,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,1
1,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,1
2,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,1
3,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,1
4,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,1


In [28]:
df.shape

(15747, 3)

In [29]:
df_combined = pd.concat([df_combined,df])
df_combined.shape

(16552, 3)

In [30]:
df = pd.read_csv("Datasets\FakeNewsNet\dataset\gossipcop_real.csv")
df['target'] = 1
df.drop(df.columns[[0,3]],axis=1,inplace=True)
df.dropna(inplace=True)
df.head()

,news_url,title,target
0,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,1
1,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,1
2,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,1
3,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,1
4,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,1


In [31]:
df_combined = pd.concat([df_combined,df])
df_fakenewsnet = df_combined
df_combined.shape

(33356, 3)

In [32]:
df_fake = pd.read_csv("Datasets/getting_real_about_fake_news/fake.csv")
df_fake.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [33]:
df_fake.shape

(12999, 20)

In [34]:
df_fake=df_fake[df_fake["language"]=="english"]

In [35]:
df_fake.shape

(12403, 20)

In [36]:
df_fake_new=df_fake.loc[:,["site_url","thread_title"]]
df_fake_new["target"]=0
df_fake_new.shape

(12403, 3)

In [37]:
df_fake_new.head()

,site_url,thread_title,target
0,100percentfedup.com,Muslims BUSTED: They Stole Millions In Gov’t B...,0
1,100percentfedup.com,Re: Why Did Attorney General Loretta Lynch Ple...,0
2,100percentfedup.com,BREAKING: Weiner Cooperating With FBI On Hilla...,0
3,100percentfedup.com,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0
4,100percentfedup.com,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0


In [38]:
df_fake_new.rename(columns = {'site_url':'news_url','thread_title':'title'}, inplace = True)
df_fake_new.head()

,news_url,title,target
0,100percentfedup.com,Muslims BUSTED: They Stole Millions In Gov’t B...,0
1,100percentfedup.com,Re: Why Did Attorney General Loretta Lynch Ple...,0
2,100percentfedup.com,BREAKING: Weiner Cooperating With FBI On Hilla...,0
3,100percentfedup.com,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0
4,100percentfedup.com,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0


In [39]:
df_combined=pd.concat([df_combined,df_fake_new])
df_combined.shape

(45759, 3)

In [57]:
train = df_combined.fillna(' ')
train['total'] = train['title']+train['news_url']
train.head()

,news_url,title,target,total
0,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,0,BREAKING: First NFL Team Declares Bankruptcy O...
1,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,0,Court Orders Obama To Pay $400 Million In Rest...
2,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,0,UPDATE: Second Roy Moore Accuser Works For Mic...
3,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,0,Oscar Pistorius Attempts To Commit Suicidehttp...
4,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,0,Trump Votes For Death Penalty For Being Gayhtt...


In [45]:
stop_words = stopwords.words('english')

In [46]:
lemmatizer = WordNetLemmatizer()

In [64]:
feature = train.loc[:,['total']]
feature.head()

,total
0,BREAKING: First NFL Team Declares Bankruptcy O...
1,Court Orders Obama To Pay $400 Million In Rest...
2,UPDATE: Second Roy Moore Accuser Works For Mic...
3,Oscar Pistorius Attempts To Commit Suicidehttp...
4,Trump Votes For Death Penalty For Being Gayhtt...


In [73]:
def lemmatize_text(text):
    filter_sentence = ''
    sentence = re.sub(r'[^\w\s]', '', text)
     # Tokenization
    words = nltk.word_tokenize(sentence)
     # Stopwords removal
    words = [w for w in words if not w in stop_words]
    for words in words:
        filter_sentence = filter_sentence  + ' ' +str(lemmatizer.lemmatize(words)).lower()
    return filter_sentence


In [75]:
train['text_lemmatized'] = train.total.apply(lemmatize_text)
train.head()

,total,target,text_lemmatized
0,muslims busted they stole millions in govt be...,0,muslim busted stole million govt benefits100p...
1,re why did attorney general loretta lynch ple...,0,attorney general loretta lynch plead fifth100...
2,breaking weiner cooperating with fbi on hilla...,0,breaking weiner cooperating fbi hillary email...
3,pin drop speech by father of daughter kidnapp...,0,pin drop speech father daughter kidnapped kil...
4,fantastic trumps 7 point plan to reform healt...,0,fantastic trump 7 point plan reform healthcar...


In [76]:
train.head(50)

,total,target,text_lemmatized
0,muslims busted they stole millions in govt be...,0,muslim busted stole million govt benefits100p...
1,re why did attorney general loretta lynch ple...,0,attorney general loretta lynch plead fifth100...
2,breaking weiner cooperating with fbi on hilla...,0,breaking weiner cooperating fbi hillary email...
3,pin drop speech by father of daughter kidnapp...,0,pin drop speech father daughter kidnapped kil...
4,fantastic trumps 7 point plan to reform healt...,0,fantastic trump 7 point plan reform healthcar...
5,hillary goes absolutely berserk on protester ...,0,hillary go absolutely berserk protester rally...
6,breaking nypd ready to make arrests in weiner...,0,breaking nypd ready make arrest weiner casehi...
7,wow whistleblower tells chilling story of mas...,0,wow whistleblower tell chilling story massive...
8,breaking clinton clearedwas this a coordinate...,0,breaking clinton clearedwas coordinated last ...
9,evil hillary supporters yell fck trumpburn tr...,0,evil hillary supporter yell fck trumpburn tru...
